<a href="https://colab.research.google.com/github/Dh0t/Tugas/blob/main/NLP_PTM12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%html
<!DOCTYPE html>
<html lang="id">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Penerjemah Sederhana</title>
    <!-- Tailwind CSS CDN -->
    <script src="https://cdn.tailwindcss.com"></script>
    <script>
        tailwind.config = {
            theme: {
                extend: {
                    fontFamily: {
                        inter: ['Inter', 'sans-serif'],
                    },
                }
            }
        }
    </script>
    <style>
        body {
            font-family: 'Inter', sans-serif;
        }
        .loading-spinner {
            border: 4px solid rgba(0, 0, 0, 0.1);
            border-left-color: #3B82F6; /* Blue-500 */
            border-radius: 50%;
            width: 24px;
            height: 24px;
            animation: spin 1s linear infinite;
            display: none; /* Hidden by default */
        }

        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
    </style>
</head>
<body class="bg-gradient-to-br from-blue-500 to-indigo-700 min-h-screen flex items-center justify-center p-4">
    <div class="bg-white rounded-lg shadow-2xl p-8 max-w-2xl w-full">
        <h1 class="text-3xl font-bold text-gray-800 mb-6 text-center">Penerjemah Sederhana</h1>
        <!-- p class="text-gray-600 mb-6 text-center">
            Terjemahkan teks menggunakan model bahasa lokal Anda dari Ollama.
            Pastikan Ollama berjalan di <code class="bg-gray-100 p-1 rounded-md">http://localhost:11434</code>.
        </p>

        <!-- Input Model Ollama -->
        <div class="mb-4">
            <label for="ollamaModel" class="block text-gray-700 text-sm font-semibold mb-2">Pilih Model :</label>
            <select id="ollamaModel" class="shadow-sm border border-gray-300 rounded-md w-full py-2 px-3 text-gray-700 leading-tight focus:outline-none focus:ring-2 focus:ring-blue-500 focus:border-blue-500">
                <option value="llama2">llama2 (Direkomendasikan)</option>
                <option value="mistral">mistral</option>
                <option value="gemma:2b">gemma:2b</option>
                <option value="other">Lainnya (Masukkan manual)</option>
            </select>
            <input type="text" id="customModel" class="mt-2 shadow-sm border border-gray-300 rounded-md w-full py-2 px-3 text-gray-700 leading-tight focus:outline-none focus:ring-2 focus:ring-blue-500 focus:border-blue-500 hidden" placeholder="Masukkan nama model kustom Anda (mis. 'codellama')">
        </div>

        <!-- Input Teks Asli -->
        <div class="mb-4">
            <label for="sourceText" class="block text-gray-700 text-sm font-semibold mb-2">Teks Asli:</label>
            <textarea id="sourceText" rows="4" class="shadow-sm border border-gray-300 rounded-md w-full py-2 px-3 text-gray-700 leading-tight focus:outline-none focus:ring-2 focus:ring-blue-500 focus:border-blue-500 resize-y" placeholder="Masukkan teks yang ingin Anda terjemahkan..."></textarea>
        </div>

        <!-- Input Bahasa Target -->
        <div class="mb-6">
            <label for="targetLanguage" class="block text-gray-700 text-sm font-semibold mb-2">Terjemahkan ke Bahasa:</label>
            <input type="text" id="targetLanguage" class="shadow-sm border border-gray-300 rounded-md w-full py-2 px-3 text-gray-700 leading-tight focus:outline-none focus:ring-2 focus:ring-blue-500 focus:border-blue-500" placeholder="Contoh: English, Spanish, Indonesian, Japanese">
        </div>

        <!-- Tombol Terjemahkan -->
        <div class="flex items-center justify-center mb-6">
            <button id="translateButton" class="bg-blue-600 hover:bg-blue-700 text-white font-bold py-2 px-6 rounded-lg shadow-md transition duration-300 ease-in-out transform hover:scale-105 flex items-center justify-center gap-2">
                Terjemahkan
                <div id="loadingSpinner" class="loading-spinner"></div>
            </button>
        </div>

        <!-- Hasil Terjemahan -->
        <div>
            <h2 class="text-xl font-bold text-gray-800 mb-3">Hasil Terjemahan:</h2>
            <div id="translationResult" class="bg-gray-50 border border-gray-200 rounded-md p-4 min-h-[100px] text-gray-800 whitespace-pre-wrap">
                Hasil terjemahan akan muncul di sini.
            </div>
            <div id="errorMessage" class="text-red-600 text-sm mt-2 hidden"></div>
        </div>
    </div>

    <script>
        document.addEventListener('DOMContentLoaded', () => {
            const sourceText = document.getElementById('sourceText');
            const targetLanguage = document.getElementById('targetLanguage');
            const translateButton = document.getElementById('translateButton');
            const translationResult = document.getElementById('translationResult');
            const errorMessage = document.getElementById('errorMessage');
            const loadingSpinner = document.getElementById('loadingSpinner');
            const ollamaModelSelect = document.getElementById('ollamaModel');
            const customModelInput = document.getElementById('customModel');

            // Toggle custom model input visibility
            ollamaModelSelect.addEventListener('change', () => {
                if (ollamaModelSelect.value === 'other') {
                    customModelInput.classList.remove('hidden');
                } else {
                    customModelInput.classList.add('hidden');
                }
            });

            translateButton.addEventListener('click', async () => {
                const text = sourceText.value.trim();
                const lang = targetLanguage.value.trim();
                let selectedModel = ollamaModelSelect.value;

                if (selectedModel === 'other') {
                    selectedModel = customModelInput.value.trim();
                }

                if (!text || !lang || !selectedModel) {
                    displayMessage('Semua kolom harus diisi!', 'error');
                    return;
                }

                // Tampilkan spinner dan nonaktifkan tombol
                loadingSpinner.style.display = 'block';
                translateButton.disabled = true;
                translationResult.textContent = 'Menerjemahkan... Mohon tunggu.';
                displayMessage('', 'clear'); // Clear previous error messages

                try {
                    const prompt = `Terjemahkan teks berikut ke dalam bahasa ${lang}: '${text}'`;
                    const ollamaUrl = 'http://localhost:11434/api/generate';

                    const payload = {
                        model: selectedModel,
                        prompt: prompt,
                        stream: false, // Kita tidak ingin streaming, hanya hasil akhir
                    };

                    const response = await fetch(ollamaUrl, {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/json',
                        },
                        body: JSON.stringify(payload),
                    });

                    if (!response.ok) {
                        const errorData = await response.text();
                        throw new Error(`Ollama API error: ${response.status} - ${errorData}`);
                    }

                    const result = await response.json();
                    let translatedText = result.response.trim();

                    // Post-processing untuk membersihkan respons dari Ollama
                    // Model sering mengulang prompt atau menambahkan informasi yang tidak perlu.
                    // Ini mungkin perlu disesuaikan tergantung model yang Anda gunakan.
                    const initialPromptPart = `Terjemahkan teks berikut ke dalam bahasa ${lang}: '${text}'`;
                    if (translatedText.startsWith(initialPromptPart)) {
                        translatedText = translatedText.replace(initialPromptPart, '').trim();
                    }
                    if (translatedText.toLowerCase().startsWith('hasil terjemahan:')) {
                        translatedText = translatedText.substring('hasil terjemahan:'.length).trim();
                    }
                    if (translatedText.toLowerCase().startsWith('the translation is:')) {
                        translatedText = translatedText.substring('the translation is:'.length).trim();
                    }
                    if (translatedText.startsWith("'") && translatedText.endsWith("'")) {
                        translatedText = translatedText.slice(1, -1);
                    }
                    if (translatedText.startsWith('"') && translatedText.endsWith('"')) {
                        translatedText = translatedhesizedText.slice(1, -1);
                    }
                    // Hapus baris kosong yang mungkin ditambahkan oleh model
                    translatedText = translatedText.split('\n').filter(line => line.trim() !== '').join('\n');


                    translationResult.textContent = translatedText || "Tidak ada terjemahan yang dihasilkan.";
                    displayMessage('Terjemahan berhasil!', 'success');

                } catch (error) {
                    console.error('Error during translation:', error);
                    let userMessage = 'Terjadi kesalahan saat menerjemahkan. ';
                    if (error.message.includes('Failed to fetch')) {
                        userMessage += 'Pastikan Ollama berjalan di http://localhost:11434 dan model yang dipilih sudah terunduh.';
                    } else {
                        userMessage += error.message;
                    }
                    translationResult.textContent = 'Gagal menerjemahkan.';
                    displayMessage(userMessage, 'error');
                } finally {
                    // Sembunyikan spinner dan aktifkan kembali tombol
                    loadingSpinner.style.display = 'none';
                    translateButton.disabled = false;
                }
            });

            function displayMessage(message, type) {
                errorMessage.textContent = message;
                if (type === 'error') {
                    errorMessage.classList.remove('hidden');
                    errorMessage.classList.remove('text-green-600');
                    errorMessage.classList.add('text-red-600');
                } else if (type === 'success') {
                    errorMessage.classList.remove('hidden');
                    errorMessage.classList.remove('text-red-600');
                    errorMessage.classList.add('text-green-600');
                } else if (type === 'clear') {
                    errorMessage.classList.add('hidden');
                }
            }
        });
    </script>
</body>
</html>